# HW5


[RQ1] Build the graph [G=(V, E)] , where V is the set of articles and E the hyperlinks among them, and provide its basic information:

    * If it is direct or not
    * The number of nodes
    * The number of edges
    * The average node degree. Is the graph dense?

In [1]:
import pandas as pd
from collections import defaultdict
import networkx as nx
import pickle

In [2]:
source_destination=pd.read_csv('wiki-topcats-reduced.txt',sep="\t",header=None,names=["source","destination"])

In [3]:
source_destination.head()

,source,destination
0,52,401135
1,52,1069112
2,52,1163551
3,62,12162
4,62,167659


In [4]:
#make a dictionary where key=source_id(article_id), value=destination_id(list of article_id's which are connected to that article)

In [5]:
source_destination_tuples=[(row["source"],row["destination"]) for idx,row in source_destination.iterrows()]

source_destination_dict=defaultdict(list)
for idx,row in source_destination.iterrows():
    source_destination_dict[row["source"]].append(row["destination"])

pickle.dump(source_destination_dict, open("source_destination_dict.p", "wb")) 

In [6]:
source_destination_dict=pickle.load(open("source_destination_dict.p", "rb"))

In [7]:
source_destination_dict[52]

[401135, 1069112, 1163551]

### The number of nodes and edges

In [8]:
#The number of nodes
Gtup = nx.Graph()
Gtup.add_edges_from(source_destination_tuples)
print("Number of nodes:",len(Gtup.nodes()),"\nNumber of edges:",len(Gtup.edges()))

Number of nodes: 461193 
Number of edges: 2174062


### Average node degree

In [9]:
nx.info(Gtup).split("\n")[4]

'Average degree:   9.4297'

###  Is the graph dense?

In [10]:
nx.density(Gtup)

2.0446319786157265e-05

In [11]:
#The graph is a sparse graph. Because the density has a small value.

In [12]:
first_key=next(iter(source_destination_dict))

(first_key,source_destination_dict[first_key])

(52, [401135, 1069112, 1163551])

### Is the graph directed or undirected?

In [13]:
nx.is_directed(Gtup)

False

In [14]:
indicatorD=False
first_key=next(iter(source_destination_dict))
first_key_tup=(first_key,source_destination_dict[first_key])
for dest in first_key_tup[1]:#check when destination is source
    if first_key not in source_destination_dict[dest]:
        indicatorD=True
if indicatorD==True:
    print("Graph is directed")
else:
    print("Graph is undirected")

Graph is directed


In [15]:
Gtup_directed = nx.DiGraph()

In [16]:
Gtup_directed.add_edges_from(source_destination_tuples)
print("Number of nodes:",len(Gtup_directed.nodes()),"\nNumber of edges:",len(Gtup_directed.edges()))

Number of nodes: 461193 
Number of edges: 2645247


In [17]:
nx.is_directed(Gtup_directed)

True

Note that there are some categories that we have removed from the graph; more precisely, you must take into account all the categories that have a number of articles greater than 3500.

In [ ]:
#Remove the categories that have a number of articles greater than 3500.

In [ ]:
categories=pd.read_csv('wiki-topcats-categories.txt',sep="\n",header=None)

In [ ]:
categories.head(2)

In [ ]:
def num_cat(row):
    cat=row
    return len(cat.split("; ")[1].split(" "))

In [ ]:
def create_df(row):
    return row[0].split("; ")

In [ ]:
categories=categories[categories.iloc[:,0].apply(lambda x:True if num_cat(x)>3500 else False)]

In [ ]:
#key=name of the category, value=list of article ids 

In [ ]:
categories_dict=defaultdict(list)
for idx in categories.index:
    cat_and_values=categories[0].loc[idx].split("; ")
    cat_name=cat_and_values[0].split(":")[1]
    categories_dict[cat_name]=list(map(int,cat_and_values[1].split(" ")))

In [ ]:
len(categories_dict["English_footballers"])

In [ ]:
#checking if it is the same    
def find_shortest_path(graph, start, end, path=[]):
        path = path + [start]
        if start == end:
            return path
        if not graph.has_key(start):
            return None
        shortest = None
        for node in graph[start]:
            if node not in path:
                newpath = find_shortest_path(graph, node, end, path)
                if newpath:
                    if not shortest or len(newpath) < len(shortest):
                        shortest = newpath
        return shortest
len(categories[0].loc[868].split("; ")[1].split(" "))

pickle.dump(categories_dict, open("categories_dict.p", "wb"))

In [ ]:
categories_dict=pickle.load(open("categories_dict.p", "rb"))

In [ ]:
len(categories_dict["English_footballers"])

In [ ]:
import collections
import math

In [ ]:
# finds shortest path between 2 nodes of a graph using BFS
def bfs_shortest_path(graph, start, goal):
    # keep track of explored nodes
    explored = []
    # keep track of all the paths to be checked
    queue = [[start]]
 
    # return path if start is goal
    if start == goal:
        return 0
 
    # keeps looping until all possible paths have been checked
    while queue:
        # pop the first path from the queue
        path = queue.pop(0)
        # get the last node from the path
        node = path[-1]
        if node not in explored:
            neighbours = graph[node]
            # go through all neighbour nodes, construct a new path and
            # push it into the queue
            for neighbour in neighbours:
                new_path = list(path)
                new_path.append(neighbour)
                queue.append(new_path)
                # return path if neighbour is goal
                if neighbour == goal:
                    return len(new_path)-1
 
            # mark node as explored
            explored.append(node)
 
    # in case there's no path between the 2 nodes
    return "So sorry, but a connecting path doesn't exist :("
 


In [ ]:
categories_dict.keys()

In [ ]:
from tqdm import tqdm

cat1 = "English_footballers"
cat2 = "The_Football_League_players"
bfs_list= []

for val1 in tqdm(categories_dict[cat1]):
    for val2 in tqdm(categories_dict[cat2]):
        bfs_list.append(bfs_shortest_path(H,val1, val2))

In [ ]:
#OUR method __?

In [ ]:
#imagine if it was ranked
#list of "ranked" categories
cat_lst=categories_dict.keys()
#make a subgraph
categories_sub=[]
for idx,cat in enumerate(cat_lst):
    categories_sub.append(categories_dict[cat])
    sub_graph = Gtup_directed.subgraph(categories_sub)
    in_edge_All=[] #sorted
    dict_previous_cat={}

    #just for first category
    if cat==cat_lst[0]:
        in_edge_temp_dict = defaultdict(int)
        for tupla in list(sub_graph.edges()):
            if tupla[1] in categories_dict[cat]:
                in_edge_temp_dict[tupla[1]] += 1
        sorted_in_edge_dict= sorted(in_edge_temp_dict.items(), key=operator.itemgetter(1), reverse= True)
        in_edge_All.append(sorted_in_edge_dict)
        dict_previous_cat=in_edge_temp_dict
    else:
        in_edge_temp_dict = defaultdict(int)
        for tupla in list(sub_graph.edges()):
            if tupla[1] in categories_dict[cat]:
                source=dict_previous_cat[tupla[0]] #previous category
                if source> 0:
                    in_edge_temp_dict[tupla[1]] += source
                else :
                    in_edge_temp_dict[tupla[1]] += 1
        dict_previous_cat=in_edge_temp_dict
        sorted_in_edge_dict= sorted(in_edge_temp_dict.items(), key=operator.itemgetter(1), reverse= True)
        in_edge_All.append(sorted_in_edge_dict)

In [ ]:
cat1 = "English_footballers"
cat2 = "The_Football_League_players"

In [ ]:
cat1_2 = categories_dict[cat1].append(categories_dict[cat2])

In [ ]:
type(categories_dict[cat1])

In [ ]:
import numpy as np

In [ ]:
H = Gtup.subgraph(categories_dict[cat1])

In [ ]:
type(list(H.edges())[0])

In [ ]:
from collections import defaultdict

In [ ]:
in_edge_dict = defaultdict(int)
for tupla in list(H.edges()):
    if tupla[1] in categories_dict[cat1]:
        in_edge_dict[tupla[1]] += 1

In [ ]:
in_edge_dict

In [ ]:
import operator

In [ ]:
sorted_in_edge_dict= sorted(in_edge_dict.items(), key=operator.itemgetter(1), reverse= True)

In [ ]:
sorted_in_edge_dict

In [ ]:
for i in sorted_in_edge_dict:
    print(i[0])

step 2

In [ ]:
H2 = Gtup_directed.subgraph(cat1_2)

In [ ]:
in_edge_dict2 = defaultdict(int)
for tupla in list(H.edges()):
    if tupla[1] in categories_dict[cat2]:
        source=in_edge_dict[tupla[0]]
        if source> 0:
            in_edge_dict2[tupla[1]] += source
        else :
            in_edge_dict2[tupla[1]] += 1
        
        

In [ ]:
in_edge_dict2

In [ ]:
sorted_in_edge_dict2= sorted(in_edge_dict2.items(), key=operator.itemgetter(1), reverse= True)

In [ ]:
pd.Series(sorted_in_edge_dict2).apply(lambda x:print(x[0]) )